In [23]:
import pandas as pd
from sklearn.metrics import confusion_matrix

In [ ]:
labelled = pd.read_excel('manual_label_sample.xlsx')
labelled

,Unnamed: 0.1,Unnamed: 0,text,timestamp,username,link,link_id,parent_id,id,subreddit_id,moderation,original text,manual
0,972545,1057760,americans hold beer oh wait ive finished drinking,2022-01-24 05:25:27,MrFantasticallyNerdy,/r/SingaporeRaw/comments/sao76t/we_are_spendin...,t3_sao76t,t3_sao76t,htzg3uf,t5_xnx04,"{'controversiality': 0, 'collapsed_reason_code...",Americans: Hold my beer. Oh wait. I've already...,0
1,13347,14500,boss relies heavily pass holders start asking ...,2022-05-06 10:50:15,SimpleReadingSG90,/r/SingaporeRaw/comments/ujhtgu/rants_of_a_19y...,t3_ujhtgu,t1_i7j8gnf,i7jextx,t5_xnx04,"{'controversiality': 0, 'collapsed_reason_code...",And the boss who relies heavily on S Pass hold...,0
2,461901,502383,mean youre paid much wipe youre tears money ea...,2022-05-08 05:02:44,OldFee3150,/r/SingaporeRaw/comments/ujddhe/chan_chun_sing...,t3_ujddhe,t3_ujddhe,i7rioq7,t5_xnx04,"{'controversiality': 0, 'collapsed_reason_code...",I mean if you’re being paid that much should j...,0
3,1094356,1190080,lol idea hes singing singapore practice sang w...,2022-09-27 06:09:30,kopisiutaidaily,/r/SingaporeRaw/comments/xonblv/communist_taxi...,t3_xonblv,t3_xonblv,iq2n4xp,t5_xnx04,"{'controversiality': 0, 'collapsed_reason_code...",Lol he has no idea what he’s singing. If Singa...,0
4,1153986,1254859,would probably change sgkl railway line opens,2023-08-09 04:33:32,TaylorFritz,/r/SingaporeRaw/comments/15m2a52/why_do_singap...,t3_15m2a52,t1_jveg4qj,jveglrl,t5_xnx04,"{'banned_at_utc': None, 'mod_reason_by': None,...",That would probably change once the SG-KL rail...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,847882,922205,yo thats night snack muffin ate last week haha...,2023-05-08 02:57:20,Rogueone65,/r/singapore/comments/13b8w1z/sq207_sin_to_mel...,t3_13b8w1z,t3_13b8w1z,jjaiifo,t5_2qh8c,"{'banned_at_utc': None, 'mod_reason_by': None,...",Yo that’s the night snack muffin I ate last we...,0
196,1377594,1498060,hello thank posting singapore seems previously...,2023-08-21 16:01:39,ModeratelyUsefulBot,/r/singapore/comments/15x9vq1/public_hospitals...,t3_15x9vq1,t3_15x9vq1,jx5avr9,t5_2qh8c,"{'banned_at_utc': None, 'mod_reason_by': None,...",Hello and thank you for posting to singapore! ...,0
197,1498756,1629859,guys celebrate vesak day too,2023-05-26 22:18:28,brownriver12,/r/singapore/comments/13spsi3/rsingapore_rando...,t3_13spsi3,t1_jlr58id,jlr5ecb,t5_2qh8c,"{'banned_at_utc': None, 'mod_reason_by': None,...",You guys celebrate vesak day too?🤣,0
198,1312718,1427653,stand upon shoulders giants walked may run ins...,2022-08-24 04:03:51,fish312,/r/singapore/comments/ww82t0/from_aug_29_masks...,t3_ww82t0,t1_iljpegw,iljvxm7,t5_2qh8c,"{'controversiality': 0, 'collapsed_reason_code...",We stand upon the shoulders of giants. She wal...,0


In [15]:
# coding: utf-8
# Author: C.J. Hutto
# Thanks to George Berry for reducing the time complexity from something like O(N^4) to O(N).
# Thanks to Ewan Klein and Pierpaolo Pantone for bringing VADER into NLTK. Those modifications were awesome.
# For license information, see LICENSE.TXT

"""
If you use the VADER sentiment analysis tools, please cite:
Hutto, C.J. & Gilbert, E.E. (2014). VADER: A Parsimonious Rule-based Model for
Sentiment Analysis of Social Media Text. Eighth International Conference on
Weblogs and Social Media (ICWSM-14). Ann Arbor, MI, June 2014.
"""
import os
import re
import math
import string
import codecs
import json
from itertools import product
from inspect import getsourcefile
from io import open

# ##Constants##

# (empirically derived mean sentiment intensity rating increase for booster words)
B_INCR = 0.293
B_DECR = -0.293

# (empirically derived mean sentiment intensity rating increase for using ALLCAPs to emphasize a word)
C_INCR = 0.733
N_SCALAR = -0.74

NEGATE = \
    ["aint", "arent", "cannot", "cant", "couldnt", "darent", "didnt", "doesnt",
     "ain't", "aren't", "can't", "couldn't", "daren't", "didn't", "doesn't",
     "dont", "hadnt", "hasnt", "havent", "isnt", "mightnt", "mustnt", "neither",
     "don't", "hadn't", "hasn't", "haven't", "isn't", "mightn't", "mustn't",
     "neednt", "needn't", "never", "none", "nope", "nor", "not", "nothing", "nowhere",
     "oughtnt", "shant", "shouldnt", "uhuh", "wasnt", "werent",
     "oughtn't", "shan't", "shouldn't", "uh-uh", "wasn't", "weren't",
     "without", "wont", "wouldnt", "won't", "wouldn't", "rarely", "seldom", "despite"]

# booster/dampener 'intensifiers' or 'degree adverbs'
# http://en.wiktionary.org/wiki/Category:English_degree_adverbs

BOOSTER_DICT = \
    {"absolutely": B_INCR, "amazingly": B_INCR, "awfully": B_INCR,
     "completely": B_INCR, "considerable": B_INCR, "considerably": B_INCR,
     "decidedly": B_INCR, "deeply": B_INCR, "effing": B_INCR, "enormous": B_INCR, "enormously": B_INCR,
     "entirely": B_INCR, "especially": B_INCR, "exceptional": B_INCR, "exceptionally": B_INCR,
     "extreme": B_INCR, "extremely": B_INCR,
     "fabulously": B_INCR, "flipping": B_INCR, "flippin": B_INCR, "frackin": B_INCR, "fracking": B_INCR,
     "fricking": B_INCR, "frickin": B_INCR, "frigging": B_INCR, "friggin": B_INCR, "fully": B_INCR,
     "fuckin": B_INCR, "fucking": B_INCR, "fuggin": B_INCR, "fugging": B_INCR,
     "greatly": B_INCR, "hella": B_INCR, "highly": B_INCR, "hugely": B_INCR,
     "incredible": B_INCR, "incredibly": B_INCR, "intensely": B_INCR,
     "major": B_INCR, "majorly": B_INCR, "more": B_INCR, "most": B_INCR, "particularly": B_INCR,
     "purely": B_INCR, "quite": B_INCR, "really": B_INCR, "remarkably": B_INCR,
     "so": B_INCR, "substantially": B_INCR,
     "thoroughly": B_INCR, "total": B_INCR, "totally": B_INCR, "tremendous": B_INCR, "tremendously": B_INCR,
     "uber": B_INCR, "unbelievably": B_INCR, "unusually": B_INCR, "utter": B_INCR, "utterly": B_INCR,
     "very": B_INCR,
     "almost": B_DECR, "barely": B_DECR, "hardly": B_DECR, "just enough": B_DECR,
     "kind of": B_DECR, "kinda": B_DECR, "kindof": B_DECR, "kind-of": B_DECR,
     "less": B_DECR, "little": B_DECR, "marginal": B_DECR, "marginally": B_DECR,
     "occasional": B_DECR, "occasionally": B_DECR, "partly": B_DECR,
     "scarce": B_DECR, "scarcely": B_DECR, "slight": B_DECR, "slightly": B_DECR, "somewhat": B_DECR,
     "sort of": B_DECR, "sorta": B_DECR, "sortof": B_DECR, "sort-of": B_DECR}

# check for sentiment laden idioms that do not contain lexicon words (future work, not yet implemented)
SENTIMENT_LADEN_IDIOMS = {"cut the mustard": 2, "hand to mouth": -2,
                          "back handed": -2, "blow smoke": -2, "blowing smoke": -2,
                          "upper hand": 1, "break a leg": 2,
                          "cooking with gas": 2, "in the black": 2, "in the red": -2,
                          "on the ball": 2, "under the weather": -2}

# check for special case idioms and phrases containing lexicon words
SPECIAL_CASES = {"the shit": 3, "the bomb": 3, "bad ass": 1.5, "badass": 1.5, "bus stop": 0.0,
                 "yeah right": -2, "kiss of death": -1.5, "to die for": 3,
                 "beating heart": 3.1, "broken heart": -2.9 }


# #Static methods# #

def negated(input_words, include_nt=True):
    """
    Determine if input contains negation words
    """
    input_words = [str(w).lower() for w in input_words]
    neg_words = []
    neg_words.extend(NEGATE)
    for word in neg_words:
        if word in input_words:
            return True
    if include_nt:
        for word in input_words:
            if "n't" in word:
                return True
    '''if "least" in input_words:
        i = input_words.index("least")
        if i > 0 and input_words[i - 1] != "at":
            return True'''
    return False


def normalize(score, alpha=15):
    """
    Normalize the score to be between -1 and 1 using an alpha that
    approximates the max expected value
    """
    norm_score = score / math.sqrt((score * score) + alpha)
    if norm_score < -1.0:
        return -1.0
    elif norm_score > 1.0:
        return 1.0
    else:
        return norm_score


def allcap_differential(words):
    """
    Check whether just some words in the input are ALL CAPS
    :param list words: The words to inspect
    :returns: `True` if some but not all items in `words` are ALL CAPS
    """
    is_different = False
    allcap_words = 0
    for word in words:
        if word.isupper():
            allcap_words += 1
    cap_differential = len(words) - allcap_words
    if 0 < cap_differential < len(words):
        is_different = True
    return is_different


def scalar_inc_dec(word, valence, is_cap_diff):
    """
    Check if the preceding words increase, decrease, or negate/nullify the
    valence
    """
    scalar = 0.0
    word_lower = word.lower()
    if word_lower in BOOSTER_DICT:
        scalar = BOOSTER_DICT[word_lower]
        if valence < 0:
            scalar *= -1
        # check if booster/dampener word is in ALLCAPS (while others aren't)
        if word.isupper() and is_cap_diff:
            if valence > 0:
                scalar += C_INCR
            else:
                scalar -= C_INCR
    return scalar


class SentiText(object):
    """
    Identify sentiment-relevant string-level properties of input text.
    """

    def __init__(self, text):
        if not isinstance(text, str):
            text = str(text).encode('utf-8')
        self.text = text
        self.words_and_emoticons = self._words_and_emoticons()
        # doesn't separate words from\
        # adjacent punctuation (keeps emoticons & contractions)
        self.is_cap_diff = allcap_differential(self.words_and_emoticons)

    @staticmethod
    def _strip_punc_if_word(token):
        """
        Removes all trailing and leading punctuation
        If the resulting string has two or fewer characters,
        then it was likely an emoticon, so return original string
        (ie ":)" stripped would be "", so just return ":)"
        """
        stripped = token.strip(string.punctuation)
        if len(stripped) <= 2:
            return token
        return stripped

    def _words_and_emoticons(self):
        """
        Removes leading and trailing puncutation
        Leaves contractions and most emoticons
            Does not preserve punc-plus-letter emoticons (e.g. :D)
        """
        wes = self.text.split()
        stripped = list(map(self._strip_punc_if_word, wes))
        return stripped

class SentimentIntensityAnalyzer(object):
    """
    Give a sentiment intensity score to sentences.
    """

    def __init__(self, lexicon_file="vader_lexicon.txt", emoji_lexicon="emoji_utf8_lexicon.txt"):
        lexicon_full_filepath = lexicon_file
        with codecs.open(lexicon_full_filepath, encoding='utf-8') as f:
            self.lexicon_full_filepath = f.read()
        self.lexicon = self.make_lex_dict()

        emoji_full_filepath = emoji_lexicon
        with codecs.open(emoji_full_filepath, encoding='utf-8') as f:
            self.emoji_full_filepath = f.read()
        self.emojis = self.make_emoji_dict()

    def make_lex_dict(self):
        """
        Convert lexicon file to a dictionary
        """
        lex_dict = {}
        for line in self.lexicon_full_filepath.rstrip('\n').split('\n'):
            if not line:
                continue
            (word, measure) = line.strip().split('\t')[0:2]
            lex_dict[word] = float(measure)
        return lex_dict

    def make_emoji_dict(self):
        """
        Convert emoji lexicon file to a dictionary
        """
        emoji_dict = {}
        for line in self.emoji_full_filepath.rstrip('\n').split('\n'):
            (emoji, description) = line.strip().split('\t')[0:2]
            emoji_dict[emoji] = description
        return emoji_dict

    def polarity_scores(self, text):
        """
        Return a float for sentiment strength based on the input text.
        Positive values are positive valence, negative value are negative
        valence.
        """
        # convert emojis to their textual descriptions
        text_no_emoji = ""
        prev_space = True
        for chr in text:
            if chr in self.emojis:
                # get the textual description
                description = self.emojis[chr]
                if not prev_space:
                    text_no_emoji += ' '
                text_no_emoji += description
                prev_space = False
            else:
                text_no_emoji += chr
                prev_space = chr == ' '
        text = text_no_emoji.strip()

        sentitext = SentiText(text)

        sentiments = []
        words_and_emoticons = sentitext.words_and_emoticons
        for i, item in enumerate(words_and_emoticons):
            valence = 0
            # check for vader_lexicon words that may be used as modifiers or negations
            if item.lower() in BOOSTER_DICT:
                sentiments.append(valence)
                continue
            if (i < len(words_and_emoticons) - 1 and item.lower() == "kind" and
                    words_and_emoticons[i + 1].lower() == "of"):
                sentiments.append(valence)
                continue

            sentiments = self.sentiment_valence(valence, sentitext, item, i, sentiments)

        sentiments = self._but_check(words_and_emoticons, sentiments)

        valence_dict = self.score_valence(sentiments, text)

        return valence_dict

    def sentiment_valence(self, valence, sentitext, item, i, sentiments):
        is_cap_diff = sentitext.is_cap_diff
        words_and_emoticons = sentitext.words_and_emoticons
        item_lowercase = item.lower()
        if item_lowercase in self.lexicon:
            # get the sentiment valence 
            valence = self.lexicon[item_lowercase]

            # check for "no" as negation for an adjacent lexicon item vs "no" as its own stand-alone lexicon item
            if item_lowercase == "no" and i != len(words_and_emoticons)-1 and words_and_emoticons[i + 1].lower() in self.lexicon:
                # don't use valence of "no" as a lexicon item. Instead set it's valence to 0.0 and negate the next item
                valence = 0.0
            if (i > 0 and words_and_emoticons[i - 1].lower() == "no") \
               or (i > 1 and words_and_emoticons[i - 2].lower() == "no") \
               or (i > 2 and words_and_emoticons[i - 3].lower() == "no" and words_and_emoticons[i - 1].lower() in ["or", "nor"] ):
                valence = self.lexicon[item_lowercase] * N_SCALAR

            # check if sentiment laden word is in ALL CAPS (while others aren't)
            if item.isupper() and is_cap_diff:
                if valence > 0:
                    valence += C_INCR
                else:
                    valence -= C_INCR

            for start_i in range(0, 3):
                # dampen the scalar modifier of preceding words and emoticons
                # (excluding the ones that immediately preceed the item) based
                # on their distance from the current item.
                if i > start_i and words_and_emoticons[i - (start_i + 1)].lower() not in self.lexicon:
                    s = scalar_inc_dec(words_and_emoticons[i - (start_i + 1)], valence, is_cap_diff)
                    if start_i == 1 and s != 0:
                        s = s * 0.95
                    if start_i == 2 and s != 0:
                        s = s * 0.9
                    valence = valence + s
                    valence = self._negation_check(valence, words_and_emoticons, start_i, i)
                    if start_i == 2:
                        valence = self._special_idioms_check(valence, words_and_emoticons, i)

            valence = self._least_check(valence, words_and_emoticons, i)
        sentiments.append(valence)
        return sentiments

    def _least_check(self, valence, words_and_emoticons, i):
        # check for negation case using "least"
        if i > 1 and words_and_emoticons[i - 1].lower() not in self.lexicon \
                and words_and_emoticons[i - 1].lower() == "least":
            if words_and_emoticons[i - 2].lower() != "at" and words_and_emoticons[i - 2].lower() != "very":
                valence = valence * N_SCALAR
        elif i > 0 and words_and_emoticons[i - 1].lower() not in self.lexicon \
                and words_and_emoticons[i - 1].lower() == "least":
            valence = valence * N_SCALAR
        return valence

    @staticmethod
    def _but_check(words_and_emoticons, sentiments):
        # check for modification in sentiment due to contrastive conjunction 'but'
        words_and_emoticons_lower = [str(w).lower() for w in words_and_emoticons]
        if 'but' in words_and_emoticons_lower:
            bi = words_and_emoticons_lower.index('but')
            for sentiment in sentiments:
                si = sentiments.index(sentiment)
                if si < bi:
                    sentiments.pop(si)
                    sentiments.insert(si, sentiment * 0.5)
                elif si > bi:
                    sentiments.pop(si)
                    sentiments.insert(si, sentiment * 1.5)
        return sentiments

    @staticmethod
    def _special_idioms_check(valence, words_and_emoticons, i):
        words_and_emoticons_lower = [str(w).lower() for w in words_and_emoticons]
        onezero = "{0} {1}".format(words_and_emoticons_lower[i - 1], words_and_emoticons_lower[i])

        twoonezero = "{0} {1} {2}".format(words_and_emoticons_lower[i - 2],
                                          words_and_emoticons_lower[i - 1], words_and_emoticons_lower[i])

        twoone = "{0} {1}".format(words_and_emoticons_lower[i - 2], words_and_emoticons_lower[i - 1])

        threetwoone = "{0} {1} {2}".format(words_and_emoticons_lower[i - 3],
                                           words_and_emoticons_lower[i - 2], words_and_emoticons_lower[i - 1])

        threetwo = "{0} {1}".format(words_and_emoticons_lower[i - 3], words_and_emoticons_lower[i - 2])

        sequences = [onezero, twoonezero, twoone, threetwoone, threetwo]

        for seq in sequences:
            if seq in SPECIAL_CASES:
                valence = SPECIAL_CASES[seq]
                break

        if len(words_and_emoticons_lower) - 1 > i:
            zeroone = "{0} {1}".format(words_and_emoticons_lower[i], words_and_emoticons_lower[i + 1])
            if zeroone in SPECIAL_CASES:
                valence = SPECIAL_CASES[zeroone]
        if len(words_and_emoticons_lower) - 1 > i + 1:
            zeroonetwo = "{0} {1} {2}".format(words_and_emoticons_lower[i], words_and_emoticons_lower[i + 1],
                                              words_and_emoticons_lower[i + 2])
            if zeroonetwo in SPECIAL_CASES:
                valence = SPECIAL_CASES[zeroonetwo]

        # check for booster/dampener bi-grams such as 'sort of' or 'kind of'
        n_grams = [threetwoone, threetwo, twoone]
        for n_gram in n_grams:
            if n_gram in BOOSTER_DICT:
                valence = valence + BOOSTER_DICT[n_gram]
        return valence

    @staticmethod
    def _sentiment_laden_idioms_check(valence, senti_text_lower):
        # Future Work
        # check for sentiment laden idioms that don't contain a lexicon word
        idioms_valences = []
        for idiom in SENTIMENT_LADEN_IDIOMS:
            if idiom in senti_text_lower:
                print(idiom, senti_text_lower)
                valence = SENTIMENT_LADEN_IDIOMS[idiom]
                idioms_valences.append(valence)
        if len(idioms_valences) > 0:
            valence = sum(idioms_valences) / float(len(idioms_valences))
        return valence

    @staticmethod
    def _negation_check(valence, words_and_emoticons, start_i, i):
        words_and_emoticons_lower = [str(w).lower() for w in words_and_emoticons]
        if start_i == 0:
            if negated([words_and_emoticons_lower[i - (start_i + 1)]]):  # 1 word preceding lexicon word (w/o stopwords)
                valence = valence * N_SCALAR
        if start_i == 1:
            if words_and_emoticons_lower[i - 2] == "never" and \
                    (words_and_emoticons_lower[i - 1] == "so" or
                     words_and_emoticons_lower[i - 1] == "this"):
                valence = valence * 1.25
            elif words_and_emoticons_lower[i - 2] == "without" and \
                    words_and_emoticons_lower[i - 1] == "doubt":
                valence = valence
            elif negated([words_and_emoticons_lower[i - (start_i + 1)]]):  # 2 words preceding the lexicon word position
                valence = valence * N_SCALAR
        if start_i == 2:
            if words_and_emoticons_lower[i - 3] == "never" and \
                    (words_and_emoticons_lower[i - 2] == "so" or words_and_emoticons_lower[i - 2] == "this") or \
                    (words_and_emoticons_lower[i - 1] == "so" or words_and_emoticons_lower[i - 1] == "this"):
                valence = valence * 1.25
            elif words_and_emoticons_lower[i - 3] == "without" and \
                    (words_and_emoticons_lower[i - 2] == "doubt" or words_and_emoticons_lower[i - 1] == "doubt"):
                valence = valence
            elif negated([words_and_emoticons_lower[i - (start_i + 1)]]):  # 3 words preceding the lexicon word position
                valence = valence * N_SCALAR
        return valence

    def _punctuation_emphasis(self, text):
        # add emphasis from exclamation points and question marks
        ep_amplifier = self._amplify_ep(text)
        qm_amplifier = self._amplify_qm(text)
        punct_emph_amplifier = ep_amplifier + qm_amplifier
        return punct_emph_amplifier

    @staticmethod
    def _amplify_ep(text):
        # check for added emphasis resulting from exclamation points (up to 4 of them)
        ep_count = text.count("!")
        if ep_count > 4:
            ep_count = 4
        # (empirically derived mean sentiment intensity rating increase for
        # exclamation points)
        ep_amplifier = ep_count * 0.292
        return ep_amplifier

    @staticmethod
    def _amplify_qm(text):
        # check for added emphasis resulting from question marks (2 or 3+)
        qm_count = text.count("?")
        qm_amplifier = 0
        if qm_count > 1:
            if qm_count <= 3:
                # (empirically derived mean sentiment intensity rating increase for
                # question marks)
                qm_amplifier = qm_count * 0.18
            else:
                qm_amplifier = 0.96
        return qm_amplifier

    @staticmethod
    def _sift_sentiment_scores(sentiments):
        # want separate positive versus negative sentiment scores
        pos_sum = 0.0
        neg_sum = 0.0
        neu_count = 0
        for sentiment_score in sentiments:
            if sentiment_score > 0:
                pos_sum += (float(sentiment_score) + 1)  # compensates for neutral words that are counted as 1
            if sentiment_score < 0:
                neg_sum += (float(sentiment_score) - 1)  # when used with math.fabs(), compensates for neutrals
            if sentiment_score == 0:
                neu_count += 1
        return pos_sum, neg_sum, neu_count

    def score_valence(self, sentiments, text):
        if sentiments:
            sum_s = float(sum(sentiments))
            # compute and add emphasis from punctuation in text
            punct_emph_amplifier = self._punctuation_emphasis(text)
            if sum_s > 0:
                sum_s += punct_emph_amplifier
            elif sum_s < 0:
                sum_s -= punct_emph_amplifier

            compound = normalize(sum_s)
            # discriminate between positive, negative and neutral sentiment scores
            pos_sum, neg_sum, neu_count = self._sift_sentiment_scores(sentiments)

            if pos_sum > math.fabs(neg_sum):
                pos_sum += punct_emph_amplifier
            elif pos_sum < math.fabs(neg_sum):
                neg_sum -= punct_emph_amplifier

            total = pos_sum + math.fabs(neg_sum) + neu_count
            pos = math.fabs(pos_sum / total)
            neg = math.fabs(neg_sum / total)
            neu = math.fabs(neu_count / total)

        else:
            compound = 0.0
            pos = 0.0
            neg = 0.0
            neu = 0.0

        sentiment_dict = \
            {"neg": round(neg, 3),
             "neu": round(neu, 3),
             "pos": round(pos, 3),
             "compound": round(compound, 4)}

        return sentiment_dict

In [16]:
analyzer = SentimentIntensityAnalyzer()

def get_sentiment_scores(text):
    return analyzer.polarity_scores(text)

# Apply the sentiment analysis function to the 'text' column
sentiment_results = labelled['text'].apply(get_sentiment_scores)

# Convert the results to a list of dictionaries
sentiment_list = sentiment_results.tolist()

# Display the sentiment list
print(sentiment_list)

[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.192, 'neu': 0.808, 'pos': 0.0, 'compound': -0.2263}, {'neg': 0.084, 'neu': 0.745, 'pos': 0.171, 'compound': 0.4215}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.213, 'neu': 0.461, 'pos': 0.325, 'compound': 0.2349}, {'neg': 0.367, 'neu': 0.633, 'pos': 0.0, 'compound': -0.8516}, {'neg': 0.08, 'neu': 0.737, 'pos': 0.183, 'compound': 0.4939}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.359, 'neu': 0.641, 'pos': 0.0, 'compound': -0.4215}, {'neg': 0.239, 'neu': 0.761, 'pos': 0.0, 'compound': -0.296}, {'neg': 0.093, 'neu': 0.731, 'pos': 0.176, 'compound': 0.3612}, {'neg': 0.438, 'neu': 0.385, 'pos': 0.177, 'compound': -0.5267}, {'neg': 0.0, 'neu': 0.635, 'pos': 0.365, 'compound': 0.8979}, {'neg': 0.0, 'neu': 0.526, 'pos': 0.474, 'compound': 0.4019}, {'neg': 0.539, 'neu': 0.288, 

In [17]:
# Now we append it to our original dataset
results_df = pd.DataFrame(sentiment_list)

labelled.reset_index(drop=True, inplace=True)
results_df.reset_index(drop=True, inplace=True)

vadar_df = pd.concat([labelled, results_df], axis = 1)
vadar_df

,Unnamed: 0.1,Unnamed: 0,text,timestamp,username,link,link_id,parent_id,id,subreddit_id,moderation,original text,manual,neg,neu,pos,compound
0,972545,1057760,americans hold beer oh wait ive finished drinking,2022-01-24 05:25:27,MrFantasticallyNerdy,/r/SingaporeRaw/comments/sao76t/we_are_spendin...,t3_sao76t,t3_sao76t,htzg3uf,t5_xnx04,"{'controversiality': 0, 'collapsed_reason_code...",Americans: Hold my beer. Oh wait. I've already...,0,0.000,1.000,0.000,0.0000
1,13347,14500,boss relies heavily pass holders start asking ...,2022-05-06 10:50:15,SimpleReadingSG90,/r/SingaporeRaw/comments/ujhtgu/rants_of_a_19y...,t3_ujhtgu,t1_i7j8gnf,i7jextx,t5_xnx04,"{'controversiality': 0, 'collapsed_reason_code...",And the boss who relies heavily on S Pass hold...,0,0.000,1.000,0.000,0.0000
2,461901,502383,mean youre paid much wipe youre tears money ea...,2022-05-08 05:02:44,OldFee3150,/r/SingaporeRaw/comments/ujddhe/chan_chun_sing...,t3_ujddhe,t3_ujddhe,i7rioq7,t5_xnx04,"{'controversiality': 0, 'collapsed_reason_code...",I mean if you’re being paid that much should j...,0,0.192,0.808,0.000,-0.2263
3,1094356,1190080,lol idea hes singing singapore practice sang w...,2022-09-27 06:09:30,kopisiutaidaily,/r/SingaporeRaw/comments/xonblv/communist_taxi...,t3_xonblv,t3_xonblv,iq2n4xp,t5_xnx04,"{'controversiality': 0, 'collapsed_reason_code...",Lol he has no idea what he’s singing. If Singa...,0,0.084,0.745,0.171,0.4215
4,1153986,1254859,would probably change sgkl railway line opens,2023-08-09 04:33:32,TaylorFritz,/r/SingaporeRaw/comments/15m2a52/why_do_singap...,t3_15m2a52,t1_jveg4qj,jveglrl,t5_xnx04,"{'banned_at_utc': None, 'mod_reason_by': None,...",That would probably change once the SG-KL rail...,0,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,847882,922205,yo thats night snack muffin ate last week haha...,2023-05-08 02:57:20,Rogueone65,/r/singapore/comments/13b8w1z/sq207_sin_to_mel...,t3_13b8w1z,t3_13b8w1z,jjaiifo,t5_2qh8c,"{'banned_at_utc': None, 'mod_reason_by': None,...",Yo that’s the night snack muffin I ate last we...,0,0.000,1.000,0.000,0.0000
196,1377594,1498060,hello thank posting singapore seems previously...,2023-08-21 16:01:39,ModeratelyUsefulBot,/r/singapore/comments/15x9vq1/public_hospitals...,t3_15x9vq1,t3_15x9vq1,jx5avr9,t5_2qh8c,"{'banned_at_utc': None, 'mod_reason_by': None,...",Hello and thank you for posting to singapore! ...,0,0.042,0.832,0.126,0.5719
197,1498756,1629859,guys celebrate vesak day too,2023-05-26 22:18:28,brownriver12,/r/singapore/comments/13spsi3/rsingapore_rando...,t3_13spsi3,t1_jlr58id,jlr5ecb,t5_2qh8c,"{'banned_at_utc': None, 'mod_reason_by': None,...",You guys celebrate vesak day too?🤣,0,0.000,0.519,0.481,0.5719
198,1312718,1427653,stand upon shoulders giants walked may run ins...,2022-08-24 04:03:51,fish312,/r/singapore/comments/ww82t0/from_aug_29_masks...,t3_ww82t0,t1_iljpegw,iljvxm7,t5_2qh8c,"{'controversiality': 0, 'collapsed_reason_code...",We stand upon the shoulders of giants. She wal...,0,0.000,0.811,0.189,0.5423


In [18]:
lst = ["ahbeng", "ahlian", "baka", "bloody hell", "bloody idiot", "bodoh", "bo liao", "buay pai seh", "buay tahan", "cb", "cb kia", "cb knn", "cb", "cb lao jia", "cb lao knn", "cb lor", "cb sia", "cb sia kia", "ccb", "chbye kia", "chao chbye", "chao chee bye", "chow chibai", "chow kar", "chow tu lan", "cibai", "dumb ass", "dumb", "fuck", "fuck you", "fking", "fucker", "fucker sia", "gila babi", "gundu", "hao lian kia", "hopeless", "idiot", "idiot", "ji bai", "jiat lat", "jialat kia", "jibai", "joker", "kan", "kan ni na", "kena sai", "kia si lang", "knn", "knn cb kia", "knnccb", "knnbccb", "kns", "kns cb", "lampa", "lan pa", "lanjiao", "lanjiao kia", "lj", "loser", "nabei", "no use kia", "noob", "pok gai", "pui", "sabo kia", "sibei jialat", "sibei sian", "si gina", "siol", "slut", "siao lang", "stupid", "suck", "sua gu", "tmd", "tiok knn", "tiok tiam", "useless", "what knn", "what the fuck", "wtf", "wu liao kia", "you die ah", "you die", "pengsan", "pong teng", "sabo", "paktor", 'Gila', 'Mampus', 'makan', 'kopi', 'teh', 'boleh', 'habis', 'lagi', 'tak', 'tak boleh', 'tak ada', 'takde', 'mesti', 'sudah', 'alamak', 'keluar', 'baik', 'siapa', 'apa', 'mana', 'banyak', 'kena', 'otah', 'otak', 'kiasu', 'kiasi', 'atas', 'bawah', 'liddat', 'suka', 'masak', 'lagi', 'boh', 'kanasai', 'jialat', 'gao', 'liak', 'pang', 'chey', 'guai', 'chio', 'gahmen', 'shiok', 'tok', 'zai', 'zhao', 'lahhh', 'wahpiang', 'heng', 'suay', 'boh liao', 'kangkang', 'chiobu', 'chapalang', 'zhu', 'gong', 'suay', 'kan', 'jian', 'buey', 'tio', 'steady bom pee pee', 'baey', 'macam', 'walaueh', 'wahlao']

In [19]:
vadar_df['toxic_0.4'] = 0
vadar_df['toxic_0.5'] = 0
vadar_df['toxic_0.6'] = 0
vadar_df['toxic_0.7'] = 0
vadar_df['toxic_0.8'] = 0

vadar_df.loc[vadar_df['compound'] <= -0.4, 'toxic_0.4'] = 1
vadar_df.loc[vadar_df['compound'] <= -0.5, 'toxic_0.5'] = 1
vadar_df.loc[vadar_df['compound'] <= -0.6, 'toxic_0.6'] = 1
vadar_df.loc[vadar_df['compound'] <= -0.7, 'toxic_0.7'] = 1
vadar_df.loc[vadar_df['compound'] <= -0.8, 'toxic_0.8'] = 1

vadar_df['toxic_0.4'] = np.where(vadar_df['text'].str.contains('|'.join(lst), case=False, na=False), 1, vadar_df['toxic_0.4'])
vadar_df['toxic_0.5'] = np.where(vadar_df['text'].str.contains('|'.join(lst), case=False, na=False), 1, vadar_df['toxic_0.5'])
vadar_df['toxic_0.6'] = np.where(vadar_df['text'].str.contains('|'.join(lst), case=False, na=False), 1, vadar_df['toxic_0.6'])
vadar_df['toxic_0.7'] = np.where(vadar_df['text'].str.contains('|'.join(lst), case=False, na=False), 1, vadar_df['toxic_0.7'])
vadar_df['toxic_0.8'] = np.where(vadar_df['text'].str.contains('|'.join(lst), case=False, na=False), 1, vadar_df['toxic_0.8'])
vadar_df

,Unnamed: 0.1,Unnamed: 0,text,timestamp,username,link,link_id,parent_id,id,subreddit_id,...,manual,neg,neu,pos,compound,toxic_0.4,toxic_0.5,toxic_0.6,toxic_0.7,toxic_0.8
0,972545,1057760,americans hold beer oh wait ive finished drinking,2022-01-24 05:25:27,MrFantasticallyNerdy,/r/SingaporeRaw/comments/sao76t/we_are_spendin...,t3_sao76t,t3_sao76t,htzg3uf,t5_xnx04,...,0,0.000,1.000,0.000,0.0000,0,0,0,0,0
1,13347,14500,boss relies heavily pass holders start asking ...,2022-05-06 10:50:15,SimpleReadingSG90,/r/SingaporeRaw/comments/ujhtgu/rants_of_a_19y...,t3_ujhtgu,t1_i7j8gnf,i7jextx,t5_xnx04,...,0,0.000,1.000,0.000,0.0000,0,0,0,0,0
2,461901,502383,mean youre paid much wipe youre tears money ea...,2022-05-08 05:02:44,OldFee3150,/r/SingaporeRaw/comments/ujddhe/chan_chun_sing...,t3_ujddhe,t3_ujddhe,i7rioq7,t5_xnx04,...,0,0.192,0.808,0.000,-0.2263,0,0,0,0,0
3,1094356,1190080,lol idea hes singing singapore practice sang w...,2022-09-27 06:09:30,kopisiutaidaily,/r/SingaporeRaw/comments/xonblv/communist_taxi...,t3_xonblv,t3_xonblv,iq2n4xp,t5_xnx04,...,0,0.084,0.745,0.171,0.4215,0,0,0,0,0
4,1153986,1254859,would probably change sgkl railway line opens,2023-08-09 04:33:32,TaylorFritz,/r/SingaporeRaw/comments/15m2a52/why_do_singap...,t3_15m2a52,t1_jveg4qj,jveglrl,t5_xnx04,...,0,0.000,1.000,0.000,0.0000,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,847882,922205,yo thats night snack muffin ate last week haha...,2023-05-08 02:57:20,Rogueone65,/r/singapore/comments/13b8w1z/sq207_sin_to_mel...,t3_13b8w1z,t3_13b8w1z,jjaiifo,t5_2qh8c,...,0,0.000,1.000,0.000,0.0000,0,0,0,0,0
196,1377594,1498060,hello thank posting singapore seems previously...,2023-08-21 16:01:39,ModeratelyUsefulBot,/r/singapore/comments/15x9vq1/public_hospitals...,t3_15x9vq1,t3_15x9vq1,jx5avr9,t5_2qh8c,...,0,0.042,0.832,0.126,0.5719,1,1,1,1,1
197,1498756,1629859,guys celebrate vesak day too,2023-05-26 22:18:28,brownriver12,/r/singapore/comments/13spsi3/rsingapore_rando...,t3_13spsi3,t1_jlr58id,jlr5ecb,t5_2qh8c,...,0,0.000,0.519,0.481,0.5719,0,0,0,0,0
198,1312718,1427653,stand upon shoulders giants walked may run ins...,2022-08-24 04:03:51,fish312,/r/singapore/comments/ww82t0/from_aug_29_masks...,t3_ww82t0,t1_iljpegw,iljvxm7,t5_2qh8c,...,0,0.000,0.811,0.189,0.5423,0,0,0,0,0


In [26]:
# Define ticker values
abs_threshold = [0.4, 0.5, 0.6, 0.7, 0.8]

# Create a list to store the results
results = []

# Iterate through the ticker values
for t in abs_threshold:
    
    # Get the actual and predicted labels
    y_true = vadar_df['manual']
    y_pred = vadar_df[f'toxic_{t}']

    # Calculate confusion matrix for 'non-toxic' vs 'toxic'
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()

    # Calculate recall
    recall = tp / (tp + fn)

    # Calculate precision
    precision = tp / (tp + fp)

    # Store the results in the list
    results.append({
        'threshold': t,
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'TN': tn,
        'precision': precision,
        'recall': recall
    })

# Convert the list of results to a DataFrame
results_df = pd.DataFrame(results)

# Display the DataFrame
print(results_df)


   threshold  TP  FP  FN   TN  precision  recall
0        0.4  20  44  20  116   0.312500   0.500
1        0.5  19  36  21  124   0.345455   0.475
2        0.6  15  35  25  125   0.300000   0.375
3        0.7  14  33  26  127   0.297872   0.350
4        0.8  12  32  28  128   0.272727   0.300
